#**How to Train YOLOv12 Object Detection Model on a Custom Dataset**

**Step 01 # Install the Ultralytics Package**

In [1]:
import os
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!nvidia-smi

# Change working directory for RunPod environment
%cd /workspace
home = os.getcwd()

%pip install ultralytics gdown
# !curl https://rclone.org/install.sh | sudo bash

Tue Nov 11 16:58:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.08             Driver Version: 550.127.08     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               On  |   00000000:D2:00.0 Off |                  Off |
| 30%   22C    P8             19W /  230W |       2MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**Step 02 # Import All the Requried Libraries**

In [2]:
import ultralytics
ultralytics.checks()

from ultralytics import YOLO
from IPython.display import Image

# Adjust dataset path for RunPod environment
dataset_path = f'/{home}/dataset'

Ultralytics 8.3.227 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA RTX A5000, 24241MiB)
Setup complete ✅ (96 CPUs, 503.5 GB RAM, 0.6/30.0 GB disk)


In [7]:
# Assuming dataset.zip is in /workspace
!gdown --fuzzy 'https://drive.google.com/file/d/1p18vaRuXzmtqNrV2udIWE8wDimLYbfRp/view?usp=sharing' -O {home}/dataset.zip
!unzip -q {home}/dataset.zip -d {home}/temp-dataset
!mv {home}/temp-dataset/*/ {dataset_path}

!echo {dataset_path}
!ls -lA {dataset_path}

Downloading...
From (original): https://drive.google.com/uc?id=1p18vaRuXzmtqNrV2udIWE8wDimLYbfRp
From (redirected): https://drive.google.com/uc?id=1p18vaRuXzmtqNrV2udIWE8wDimLYbfRp&confirm=t&uuid=f30d006d-5213-4b04-97a6-e108aee4b6bf
To: /workspace/dataset.zip
100%|██████████████████████████████████████| 1.84G/1.84G [00:39<00:00, 46.7MB/s]
//workspace/dataset
total 6845
drwxrwxrwx 4 root root 2003987 Nov 10 16:57 test
drwxrwxrwx 4 root root 3000166 Nov 10 16:57 train
drwxrwxrwx 4 root root 2004146 Nov 10 16:57 valid


**Step # 03 Fine-tune YOLOv12 model on a Custom Dataset**
We are now ready to fine-tune our YOLOv12 model. In the code below, we initialize the model using a starting checkpoint—here, we use `yolov12s.yaml`, but you can replace it with any other model (e.g., `yolov12n.pt`, `yolov12m.pt`, `yolov12l.pt`, or `yolov12x.pt`) based on your preference. We set the training to run for 50 epochs in this example; however, you should adjust the number of epochs along with other hyperparameters such as batch size, image size, and augmentation settings (scale, mosaic, mixup, and copy-paste) based on your hardware capabilities and dataset size.



**Note:** **Note that after training, you might encounter a `TypeError: argument of type 'PosixPath' is not iterable error` — this is a known issue, but your model weights will still be saved, so you can safely proceed to running inference.**

In [9]:
# choose one:

# model = YOLO("yolo12n.pt")
# model = YOLO("yolo12s.pt")
# model = YOLO("yolo12m.pt")
# model = YOLO("yolo12l.pt")
model = YOLO("yolo12x.pt")

results = model.train(
    data=f'{dataset_path}/data.yaml',
    batch=64,
    # cos_lr=True,
    # lr0=0.005,
    # optimizer="AdamW",
    # weight_decay=0.01,
    # momentum=0.09,
    epochs=50,
    # patience=80,
    # deterministic=False,
    # warmup_epochs=3,
    # warmup_bias_lr=0.1,
    # iou=0.75,

    # auto_augment="randaugment",
    # mosaic=0.4,
    # mixup=0.1,
    # close_mosaic=10,
    # copy_paste=0.1,
    # erasing=0.4,
    # hsv_h=0.015, hsv_s=0.5, hsv_v=0.4,
    # scale=0.2,
    # translate=0.1,
    # degrees=15,
    # flipud=0, # disable
)

Ultralytics 8.3.227 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA RTX A5000, 24241MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=//workspace/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=Tru

OutOfMemoryError: CUDA out of memory. Tried to allocate 76.00 MiB. GPU 0 has a total capacity of 23.67 GiB of which 26.25 MiB is free. Process 635165 has 23.61 GiB memory in use. Of the allocated memory 23.17 GiB is allocated by PyTorch, and 127.91 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [6]:
train_nr = ''

best_model_path = f'{home}/runs/detect/train{train_nr}/weights/best.pt'

best_model = YOLO(best_model_path)

# Validate the model

metrics = best_model.val()  # no arguments needed, dataset and settings remembered
results = best_model.predict(source = f"{dataset_path}/test/images", save = True)

Ultralytics 8.3.227 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA RTX A5000, 24241MiB)
YOLOv12n summary (fused): 159 layers, 2,557,508 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.8±0.1 ms, read: 19.3±12.1 MB/s, size: 100.3 KB)
val: Scanning /workspace/dataset/valid/labels.cache... 402 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 402/402 25.2Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 26/26 6.8it/s 3.8s
                   all        402        759      0.658       0.59       0.63      0.406
                   boy        218        278      0.616      0.586      0.622       0.42
                  girl        202        259      0.659      0.568      0.618      0.389
                   man         85         90      0.737       0.53      0.627      0.416
                 woman        112        132      0.621      0.674      0.653      0.401
Speed: 1.0ms preprocess, 2.8ms inferen

**Step # 04 Evaluate fine-tuned YOLOv12 model**

In [ ]:
!ls {home}/runs/detect/train{train_nr}/

In [ ]:
Image(filename=f'{home}/runs/detect/train{train_nr}/confusion_matrix.png', width=1000)

In [ ]:
Image(filename=f'{home}/runs/detect/train{train_nr}/confusion_matrix_normalized.png', width=1000)

In [ ]:
Image(filename=f'{home}/runs/detect/train{train_nr}/results.png', width=1000)

Precision = TP / (TP + FP)

Precision is simply true positives out of total detections.

In [ ]:

Image(filename=f'{home}/runs/detect/train{train_nr}/P_curve.png', width=1000)

Recall = TP / (TP + FN)



Recall is the True Positive out of all Ground Truths

In [ ]:
Image(filename=f'{home}/runs/detect/train{train_nr}/R_curve.png', width=1000)

In [ ]:
Image(filename=f'{home}/runs/detect/train{train_nr}/train_batch0.jpg', width=1000)

In [ ]:
Image(filename=f'{home}/runs/detect/train{train_nr}/val_batch0_pred.jpg', width=1000)

In [ ]:
Image(filename=f'{home}/runs/detect/train{train_nr}/val_batch1_pred.jpg', width=1000)